In [1]:
# !pip install tensorflow
# !pip install Keras-Preprocessing


In [2]:
import os
import pandas as pd
import numpy as np

from pyvi.ViTokenizer import ViTokenizer #vietnamese tokenizer

import re #regular expression

In [3]:
import tensorflow as tf
# from keras.preprocessing import text, sequence, tokenizer

In [4]:
from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D, Reshape, Dropout, Concatenate
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

from keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D, Reshape, Flatten, Dropout, Concatenate

In [5]:
import pickle
from keras_preprocessing.sequence import pad_sequences

# from tensorflow.keras.models import Sequential 

from tensorflow.keras import layers, Sequential

In [6]:
DIR_DATASET = os.path.join('dataset')

PATH_TRAIN = os.path.join(DIR_DATASET, 'core-data\\train.csv')
PATH_DEV = os.path.join(DIR_DATASET, 'core-data\dev.csv')
PATH_TEST = os.path.join(DIR_DATASET, 'core-data\\test.csv')

EMBEDDING_PATH = os.path.join(DIR_DATASET, 'cc.vi.300.vec')
TOKENIZER_PATH = os.path.join(DIR_DATASET, 'tokenizer.pickle')
VISTOPWORDS_PATH = os.path.join(DIR_DATASET, "vietnamese-stopwords.txt")

MODEL_DIR = 'model'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [7]:
data_train = pd.read_csv(PATH_TRAIN)

data_dev = pd.read_csv(PATH_DEV)

data_test = pd.read_csv(PATH_TEST)

In [8]:
with open(VISTOPWORDS_PATH, "r", encoding="utf-8") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip('\n')
        stopwords.append(dd)
    stopwords = set(stopwords)

In [9]:
# read csv data
data_train = pd.read_csv(PATH_TRAIN)
data_dev = pd.read_csv(PATH_DEV)
data_test = pd.read_csv(PATH_TEST)

In [10]:
data_train.columns

Index(['link', 'rating', 'comment', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'label', 'spam_label'],
      dtype='object')

In [11]:
data_train=data_train.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)
data_dev=data_dev.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)
data_test=data_test.drop(['link', 'rating', 'categories', 'category', 'product_name',
       'description', 'num_sold', 'num_review', 'spam_label'], axis=1)

In [12]:
data_train.head(10)

,comment,label
0,sản phẩm đúng quảng cáo. giao hàng hơi chậm nh...,0
1,"Mình nhận hàng ngày hôm qua, tiki giao hàng đú...",0
2,"Máy giặt quá tốt, chỉ có 5 tr được cái máy giặ...",0
3,chất liệu vải tốt mát nói chung là chất lượng ...,0
4,Tủ tốt giá tốt nhiều hơn so với các trung tâm...,0
5,Mọi người dưới 56kg nên mua sz S thoii nha. Fo...,1
6,giao nhanh đóng hàng cẩn thận ăn ngon lắm,0
7,10 điểm. Giao hàng nhanh. Cầu hình cao màn hìn...,0
8,Bé nhà mình tiêu thụ loại này tốt!! Tuy nhiên ...,0
9,Ỏ Bến Tre mà vừa đặt hàng hôm qua nay nhận dc ...,0


In [13]:
dup_train = data_train.duplicated().sum()
dup_dev = data_dev.duplicated().sum()
dup_test = data_test.duplicated().sum()

print("train's duplicates: ", dup_train,
    "\ndev's duplicates: ", dup_dev, "\ntest's duplicates: ", dup_test)

train's duplicates:  8 
dev's duplicates:  0 
test's duplicates:  3


In [14]:
data_train=data_train.drop_duplicates(keep='first')
data_dev=data_dev.drop_duplicates(keep='first')
data_test=data_test.drop_duplicates(keep='first')

In [15]:
dup_train = data_train.duplicated().sum()
dup_dev = data_dev.duplicated().sum()
dup_test = data_test.duplicated().sum()

print("train's duplicates: ", dup_train,
    "\ndev's duplicates: ", dup_dev, "\ntest's duplicates: ", dup_test)

train's duplicates:  0 
dev's duplicates:  0 
test's duplicates:  0


PREPROCESS FUNCTION

In [16]:
# filder stop word
def filter_stop_words(train_sentences, stop_words):
    new_sent = [word for word in train_sentences.split() if word not in stop_words]
    train_sentences = ' '.join(new_sent)
    return train_sentences

# remove emoji
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)
# declare func and adding some remove
def preprocess(text, tokenized = True, lowercased = True):
    text = ViTokenizer.tokenize(text) if tokenized else text
    text = filter_stop_words(text, stopwords)
    text = deEmojify(text)
    text = text.lower() if lowercased else text

    text = text.strip()
    text = re.compile('<.*?>').sub('', text)
    text = re.sub('\s+', ' ', text) 
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text

def pre_process_features(X, y, tokenized = True, lowercased = True):
    X = np.array(X)
    y = np.array(y)
    X = [preprocess(str(p), tokenized = tokenized, lowercased = lowercased) for p in list(X)]
    for idx, ele in enumerate(X):
        if not ele:
            X = np.delete(X, idx)
            y = np.delete(y, idx)
    return X, y


In [17]:
text_01 = "[HN] Hiện tại bên mình vẫn đang cần tuyển vị trí C# developer (.NET framework, C# biết winforms (2-3 năm)) sẵn sàng làm thêm FE hoặc Python, Java nếu được giao. Lương thỏa thuận, có 13-15 tháng lương. Quan tâm ib lấy JD."
text_02 = preprocess(text_01)
print("After preprocess: ", text_01, "\n")
print("Before preprocess: ", text_02)

After preprocess:  [HN] Hiện tại bên mình vẫn đang cần tuyển vị trí C# developer (.NET framework, C# biết winforms (2-3 năm)) sẵn sàng làm thêm FE hoặc Python, Java nếu được giao. Lương thỏa thuận, có 13-15 tháng lương. Quan tâm ib lấy JD. 

Before preprocess:   hn hiện_tại tuyển vị_trí c developer net framework c winforms sẵn_sàng fe python java giao lương thỏa_thuận lương quan_tâm ib jd


PREPROCESS

In [18]:
data_train

,comment,label
0,sản phẩm đúng quảng cáo. giao hàng hơi chậm nh...,0
1,"Mình nhận hàng ngày hôm qua, tiki giao hàng đú...",0
2,"Máy giặt quá tốt, chỉ có 5 tr được cái máy giặ...",0
3,chất liệu vải tốt mát nói chung là chất lượng ...,0
4,Tủ tốt giá tốt nhiều hơn so với các trung tâm...,0
...,...,...
14301,dùng okee ko gây nóng máy nhưng chất của ốp om...,0
14302,"Rất tốt, rất đáng tiền, nằm rất êm, lúc mới mu...",0
14303,"giao đủ, đóng gói chắc chắn, giao hàng nhanh \...",1
14304,giao hỏa tốc rẻ và nhanh. Nhưng trước đó mình ...,0


In [19]:
# slit data so that X_train is review and y_train is label
X_train = data_train.iloc[:, 0:1]
y_train = data_train.iloc[:, 1:2]

X_dev = data_train.iloc[:, 0:1]
y_dev = data_train.iloc[:, 1:2]

X_test = data_train.iloc[:, 0:1]
y_test = data_train.iloc[:, 1:2]    

In [20]:
y_train

,label
0,0
1,0
2,0
3,0
4,0
...,...
14301,0
14302,0
14303,1
14304,0


In [21]:
# transform X_train into narray (train_X) and simular
train_X, train_y= pre_process_features(X_train['comment'], y_train['label'], tokenized=True, lowercased = True)
dev_X, dev_y= pre_process_features(X_dev['comment'], y_dev['label'], tokenized=True, lowercased = True)
test_X, test_y = pre_process_features(X_test['comment'], y_test['label'], tokenized=True, lowercased = True)

In [22]:
train_X.shape

(14271,)

In [23]:
train_y.shape

(14271,)

DECLARE FUNCTIONS

In [24]:
embeddings_index = {}
with open(EMBEDDING_PATH, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [25]:
tokenizer = text.Tokenizer(lower=False, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~\t\n')
tokenizer.fit_on_texts(train_X)

In [26]:
with open(TOKENIZER_PATH, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [27]:
def make_featues(X, y, tokenizer, is_one_hot_label=True, number_class1=2):
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=sequence_length)
    if is_one_hot_label: 
        y = to_categorical(y, num_classes=number_class1)
    return X, y

In [28]:
vocabulary_size = 10000
sequence_length = 100
maxlen = 100

embedding_dim = 300
batch_size = 256
epochs = 40
drop = 0.5

# filter_sizes = [2,3,5]
# num_filters = 32

filter_sizes = [2,3,5]
num_filters = 32

word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= vocabulary_size:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [29]:
vocabulary_size = 10000
sequence_length = 100

embedding_dim = 300
batch_size = 256
epochs = 40
drop = 0.5

filter_sizes = [2,3,5]
num_filters = 32

word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i >= vocabulary_size:
        continue

    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [30]:
train_XX, train_yy = make_featues(train_X, train_y, tokenizer)
dev_XX, dev_yy = make_featues(dev_X, dev_y, tokenizer)
test_XX, test_yy = make_featues(test_X, test_y, tokenizer, is_one_hot_label=False)

In [31]:
train_XX.shape

(14271, 100)

In [32]:
print(train_XX[5, :])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0  268  141    3  710 6622   18
  347   49  414  497   13   65   82 6623    4    7  711   37  119 2741
  498  831]


In [33]:
train_yy[6, :]

array([1., 0.])

In [34]:
print(type(train_XX), train_XX.shape)
print(type(train_yy), train_yy.shape)
print(type(dev_XX), dev_XX.shape)
print(type(dev_yy), dev_yy.shape)
print(type(test_XX), test_XX.shape)
print(type(test_yy), test_yy.shape)

<class 'numpy.ndarray'> (14271, 100)
<class 'numpy.ndarray'> (14271, 2)
<class 'numpy.ndarray'> (14271, 100)
<class 'numpy.ndarray'> (14271, 2)
<class 'numpy.ndarray'> (14271, 100)
<class 'numpy.ndarray'> (14271,)


In [35]:
train_XX.shape

(14271, 100)

CNNs MODEL

In [36]:
# TextCNN
# Add layers
# textCNNModel = Sequential()
# textCNNModel.add(layers.Embedding(num_words, embedding_dim, input_length=sequence_length))
# # textCNNModel.add(layers.SpatialDropout1D(0.2))
# textCNNModel.add(layers.Conv1D(128, 5, activation='relu'))
# textCNNModel.add(layers.GlobalMaxPooling1D())
# textCNNModel.add(layers.Dense(256, activation='relu'))
# textCNNModel.add(layers.Dense(1, activation='softmax'))  # Change to match the number of classes
# textCNNModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Change loss function
# textCNNModel.summary()

# textCNNModel = Sequential()
# textCNNModel.add(layers.Embedding(num_words, embedding_dim, input_length=sequence_length))
# textCNNModel.add(layers.Conv1D(128, 5, activation='relu'))
# textCNNModel.add(layers.GlobalMaxPooling1D())
# textCNNModel.add(layers.Dense(256, activation='relu'))
# textCNNModel.add(layers.Dense(1, activation='sigmoid'))  # Changed to 'sigmoid' activation, 1 neuron
# textCNNModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Changed loss function

textCNNModel = Sequential()
textCNNModel.add(layers.Embedding(num_words, embedding_dim, input_length=sequence_length))
textCNNModel.add(layers.Conv1D(128, 5, activation='relu'))
textCNNModel.add(layers.GlobalMaxPooling1D())
textCNNModel.add(layers.Dense(256, activation='relu'))
textCNNModel.add(layers.Dense(2, activation='softmax'))  # Changed to 2 neurons for two classes
textCNNModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Keep loss function


E:\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [37]:
# Test for model accuracy
textCNNModel.fit( train_XX, train_yy,
    epochs=10,
    verbose=2,
    validation_data=(dev_XX, dev_yy),
    batch_size=batch_size)

Epoch 1/10
56/56 - 17s - 295ms/step - accuracy: 0.7795 - loss: 0.4806 - val_accuracy: 0.8729 - val_loss: 0.3166
Epoch 2/10
56/56 - 12s - 222ms/step - accuracy: 0.8817 - loss: 0.2970 - val_accuracy: 0.9421 - val_loss: 0.1808
Epoch 3/10
56/56 - 14s - 249ms/step - accuracy: 0.9416 - loss: 0.1600 - val_accuracy: 0.9762 - val_loss: 0.0797
Epoch 4/10
56/56 - 13s - 232ms/step - accuracy: 0.9730 - loss: 0.0773 - val_accuracy: 0.9881 - val_loss: 0.0384
Epoch 5/10
56/56 - 12s - 214ms/step - accuracy: 0.9868 - loss: 0.0416 - val_accuracy: 0.9903 - val_loss: 0.0342
Epoch 6/10
56/56 - 12s - 216ms/step - accuracy: 0.9905 - loss: 0.0271 - val_accuracy: 0.9954 - val_loss: 0.0160
Epoch 7/10
56/56 - 12s - 210ms/step - accuracy: 0.9941 - loss: 0.0190 - val_accuracy: 0.9963 - val_loss: 0.0120
Epoch 8/10
56/56 - 11s - 204ms/step - accuracy: 0.9954 - loss: 0.0149 - val_accuracy: 0.9969 - val_loss: 0.0111
Epoch 9/10
56/56 - 12s - 219ms/step - accuracy: 0.9948 - loss: 0.0159 - val_accuracy: 0.9954 - val_loss:

In [38]:
train_XX.shape

(14271, 100)

In [39]:
train_yy.shape

(14271, 2)

In [40]:
train_y_binary = train_yy[:, 0]
dev_y_binary = dev_yy[:, 0]

In [41]:
train_y_binary

array([1., 1., 1., ..., 0., 1., 1.])

In [42]:
# Evaluate the model on the TEST data
loss, accuracy = textCNNModel.evaluate(test_XX, test_yy)
print(f"LOSS: {loss}")
print ('Accuracy '+ str(accuracy))

ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None,), output.shape=(None, 2)